In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Cleaning the data

In [ ]:
df = pd.read_csv('Financial Distress.csv')
print(df.isnull().sum())
df = df.groupby('Company').mean(numeric_only=True).reset_index()
df = df.drop(columns=['Time'])
print(df.head(5))
print(len(df))

### Vizualize tha data or sumn

In [ ]:
# Plotting the distribution of Financial Distress
# Splitting values into bins with step of 0.5
bins = np.arange(min(df['Financial Distress']), max(df['Financial Distress']) + 0.5, 0.5)

# Show histogram
plt.figure(figsize=(8, 5))
plt.hist(df['Financial Distress'], bins=bins, color="darkred", edgecolor='black', alpha=0.7)
plt.title(f"Distribution of Financial Distress: x ∈ [{min(df['Financial Distress']):.2f}, {max(df['Financial Distress']):.2f}]")
plt.xlabel('Financial Distress')
plt.ylabel('Number of Companies')
plt.show()



